### Imports

In [0]:
%tensorflow_version 1

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [0]:
import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
import math

Using TensorFlow backend.


In [0]:
mnist_data = input_data.read_data_sets('MNIST_data/', one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


### Data Examination

### Tensorboard

In [0]:
# Getting ngrok to service a local server to internet
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2020-06-06 10:32:30--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.214.214.70, 54.159.115.94, 52.70.230.150, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.214.214.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.3’

ngrok-stable-linux- 100%[===================>]  13.13M  5.30MB/s    in 2.5s    

2020-06-06 10:32:34 (5.30 MB/s) - ‘ngrok-stable-linux-amd64.zip.3’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
# Log directory to write training logs like loss
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'.format(LOG_DIR)
)


In [0]:
# setting up my server
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
# setting up my server
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


https://25bb7da230b6.ngrok.io


In [0]:
# code we wrote in class, this didnt work in class
# ! curl -s http://localhost:4040/api/tunnels | python3 -c \
# "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
# Getting a tensorboard object, I need to pass it the log dir path
from keras.callbacks import TensorBoard
tbCallBack = TensorBoard(log_dir = './log',
                         histogram_freq = 1,
                         write_graph = True,
                         write_grads = True,
                         write_images = True)

### Network

In [0]:
# I use keras functional API here.
# I need to define an Input layer
# I pass variables from previous layers to next layers


from keras.layers import Conv2D, BatchNormalization, Dropout, Input, Dense, Flatten

inp = Input(shape = (28,28,1))

y1 = Conv2D(filters  = 100,
            kernel_size = (5,5),
            strides = (1,1),
            padding = 'same',
            activation = 'relu',
            use_bias = True,
            kernel_initializer = 'glorot_uniform')(inp)

y2 = Conv2D(filters  = 25,
            kernel_size = (5,5),
            strides = (1,1),
            padding = 'same',
            activation = 'relu',
            use_bias = True,
            kernel_initializer = 'glorot_uniform')(y1)

y2_flat = Flatten()(y2)

pred = Dense(10,activation = 'softmax')(y2_flat)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
from keras.models import Model
from keras.optimizers import Adam

# This is the place where we actually created the model
# you can output multiple outputs, in a list
inzNet = Model(input = inp, outputs = pred)

# our optimizer
opt = Adam(lr = 0.001)

# we compile to start the weights etc.
inzNet.compile(optimizer = opt,
               loss = 'categorical_crossentropy',
               metrics = ['accuracy'])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  after removing the cwd from sys.path.


In [0]:
dataset_size = 10000
batch_size = 128
input_data , gt_data = mnist_data.train.next_batch(dataset_size)
input_data = input_data.reshape((dataset_size,28,28,1))

# fit method is our training loop in keras
# similar methods: train_on_batch, test_on_batch, predict, evaluate
# all methods I wrote above perform forward prop. some, do not perform backprop.
inzNet.fit(input_data,gt_data,
           batch_size=batch_size,
           epochs =4,
           validation_split = 0.1,
           callbacks = [tbCallBack]
           )



Train on 9000 samples, validate on 1000 samples




Epoch 1/4
9000/9000 [==============================] - 3s 377us/step - loss: 0.5103 - accuracy: 0.8379 - val_loss: 0.2038 - val_accuracy: 0.9310

Epoch 2/4
9000/9000 [==============================] - 1s 86us/step - loss: 0.1320 - accuracy: 0.9593 - val_loss: 0.1025 - val_accuracy: 0.9640
Epoch 3/4
9000/9000 [==============================] - 1s 88us/step - loss: 0.0719 - accuracy: 0.9773 - val_loss: 0.0885 - val_accuracy: 0.9720
Epoch 4/4
9000/9000 [==============================] - 1s 89us/step - loss: 0.0479 - accuracy: 0.9844 - val_loss: 0.0877 - val_accuracy: 0.9710


In [0]:
sample_size = 5000
input_test, gt_test = mnist_data.test.next_batch(sample_size)
input_test = input_test.reshape((sample_size,28,28,1))

inzNet.evaluate(input_test,gt_test)

5000/5000 [==============================] - 0s 94us/step


[0.07531226598940557, 0.978600025177002]